# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# !pip install pymongo

In [2]:
from pymongo import MongoClient
import json

In [3]:
# Server connection
client = MongoClient("localhost:27017")

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
# Mostrar los nombres de las bases de datos existentes
client.list_database_names()

['Ironhack', 'admin', 'config', 'ironhacktest', 'local']

In [6]:
# Connection to the database

# Para crear una conexión con la base de datos en la que vayamos a trabajar
db = client.get_database("Ironhack")

In [7]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [8]:
db.list_collection_names()

['companies']

In [9]:
collection = db.get_collection("companies")

In [10]:
collection.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'acquisition': {'acquired_day': 16,
  'acquired_month': 12,
  'acquired_year': 2013,
  'acquiring_company': {'name': 'Viggle', 'permalink': 'viggle'},
  'price_amount': 30000000,
  'price_currency_code': 'USD',
  'source_description': ' Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint',
  'source_url': 'http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet',
  'term_code': 'cash_and_stock'},
 'acquisitions': [],
 'alias_list': '',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'blog_url': 'http://digitalquarters.net/',
 'category_code': 'web',
 'competitions': [{'competitor': {'name': 'Wikia', 'permalink': 'wikia'}},
  {'competitor': {'name': 'JotSpot', 'permalink': 'jotspot'}},
  {'competitor': {'name': 'Socialtext', 'permalink': 'socialtext'}},
  {'competitor': {'name': 'Ning by Glam Media', 'permalink': 'ning'}},
  {'competitor': {'name': 'Soceeo', 'permal

In [11]:
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [20]:
res = list(collection.find({'name': 'Babelgum'}))
print(len(res))

1


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [21]:
res = list(collection.find({'number_of_employees': { '$gt': 5000 } } ).sort('number_of_employees',1).limit(20))

print(len(res))
print(res[0])

20
{'_id': ObjectId('52cdef7c4bab8bd675297e49'), 'name': 'Nintendo', 'permalink': 'nintendo', 'crunchbase_url': 'http://www.crunchbase.com/company/nintendo', 'homepage_url': 'http://nintendo.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': 'NintendoAmerica', 'category_code': 'games_video', 'number_of_employees': 5080, 'founded_year': 1889, 'founded_month': 9, 'founded_day': 23, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': 'games, console', 'alias_list': '', 'email_address': 'noalegal@noa.nintendo.com', 'phone_number': '1-800-255-3700 ', 'description': '', 'created_at': 'Sat Dec 22 13:27:00 UTC 2007', 'updated_at': 'Thu Sep 05 12:41:16 UTC 2013', 'overview': '<p>Nintendo, a technology company widely known for its line of game consoles, was actually founded in 1889 by Fusajiro Yamauchi. It began as a card game company and evolved into one of the largest Japanese companies with a Market Capitalization of over 

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [22]:
proj = {'_id': 0,'name': 1, 'founded_year': 1}
res = list(collection.find( { 'founded_year': { '$gte': 2000, '$lte': 2005 } },proj ))

print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))

3734
{
    "founded_year": 2005,
    "name": "Wetpaint"
}


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [23]:
proj = {'_id': 0,'name': 1, 'ipo': 1}
res = list(collection.find( {'ipo': {'$exists': 'true' },
                             'ipo.valuation_amount': {'$exists': 'true' , '$type': 'int', '$gte': 100000000},
                             'founded_year': { '$lte': 2010 } }, proj ))
print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))

28
{
    "ipo": {
        "pub_day": 2,
        "pub_month": 3,
        "pub_year": 2012,
        "stock_symbol": "NYSE:YELP",
        "valuation_amount": 1300000000,
        "valuation_currency_code": "USD"
    },
    "name": "Yelp"
}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [24]:
res = list(collection.find({'number_of_employees': { '$lte': 1000 } , 
                           'founded_year': { '$lte': 2005 }} ).sort('number_of_employees',1).limit(10))

print(len(res))
print(res[0])

10
{'_id': ObjectId('52cdef7c4bab8bd675297d93'), 'name': 'Fox Interactive Media', 'permalink': 'fox-interactive-media', 'crunchbase_url': 'http://www.crunchbase.com/company/fox-interactive-media', 'homepage_url': 'http://www.newscorp.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': 'twitterapi', 'category_code': 'web', 'number_of_employees': 0, 'founded_year': 1979, 'founded_month': 6, 'founded_day': 1, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': '', 'tag_list': '', 'alias_list': None, 'email_address': '', 'phone_number': '', 'description': '', 'created_at': 'Thu May 31 21:46:57 UTC 2007', 'updated_at': 'Mon Aug 19 17:13:27 UTC 2013', 'overview': '<p>Fox Interactive Media (FIM) oversees <a href="http://www.crunchbase.com/company/newscorporation" title="News Corporation">News Corporation</a>&#8217;s Internet business operations.</p>', 'image': {'available_sizes': [[[150, 71], 'assets/images/resized/0001/0824/10824v1-max-150x

### 6. All the companies that don't include the `partners` field.

In [46]:
res = list(collection.find({'partners': {'$exists': False}}))
print(len(res))

0


### 7. All the companies that have a null type of value on the `category_code` field.

In [47]:
res = list(collection.find({ 'category_code' : 'null'}) )
# res = list(collection.find({ 'category_code' : { '$type': 10 } })
# res = list(collection.find({ 'category_code' : { '$eq': null } } )
# res = list(collection.find({ 'category_code' : { '$ne': null } } )           

print(len(res))
print('Same query in MongoDB Compass shows 2751 document')

0
Same query in MongoDB Compass shows 2751 document


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [32]:
proj = {'_id': 0,'name': 1, 'number_of_employees': 1}
res = list(collection.find( { 'number_of_employees': { '$gte': 100, '$lt': 1000 } },proj ))

print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))


917
{
    "name": "AdventNet",
    "number_of_employees": 600
}


### 9. Order all the companies by their IPO price in a descending order.

In [33]:
res = list(collection.find( {'ipo': {'$exists': 'true' },
                             'ipo.valuation_amount': {'$exists': 'true' , '$type': 'int'} }).sort('ipo.valuation_amount',-1))
print(len(res))
print(res[0])

47
{'_id': ObjectId('52cdef7c4bab8bd675297f15'), 'name': 'Nielsen', 'permalink': 'nielsen', 'crunchbase_url': 'http://www.crunchbase.com/company/nielsen', 'homepage_url': 'http://nielsen.com', 'blog_url': 'http://blog.nielsen.com/nielsenwire', 'blog_feed_url': '', 'twitter_username': 'nielsenwire', 'category_code': 'web', 'number_of_employees': None, 'founded_year': 1922, 'founded_month': None, 'founded_day': None, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': '', 'alias_list': '', 'email_address': 'ContactCommunications@nielsen.com', 'phone_number': ' 800.864.1224', 'description': '', 'created_at': 'Wed Aug 22 10:57:37 UTC 2007', 'updated_at': 'Tue Nov 26 03:11:23 UTC 2013', 'overview': '<p>Nielsen is a leading global provider of information and insights into what consumers watch and buy.</p>\n\n<p>Nielsen is a information and measurement company with leading market positions in marketing and consumer information, televi

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [34]:
proj = {'_id': 0,'name': 1, 'number_of_employees': 1}
res = list(collection.find({'number_of_employees' : { '$exists': 'true' }},proj).sort('number_of_employees', -1).limit(10))

print(json.dumps(res, indent=4, sort_keys=True))

[
    {
        "name": "Siemens",
        "number_of_employees": 405000
    },
    {
        "name": "IBM",
        "number_of_employees": 388000
    },
    {
        "name": "Toyota",
        "number_of_employees": 320000
    },
    {
        "name": "PayPal",
        "number_of_employees": 300000
    },
    {
        "name": "Nippon Telegraph and Telephone Corporation",
        "number_of_employees": 227000
    },
    {
        "name": "Samsung Electronics",
        "number_of_employees": 221726
    },
    {
        "name": "Accenture",
        "number_of_employees": 205000
    },
    {
        "name": "Tata Consultancy Services",
        "number_of_employees": 200300
    },
    {
        "name": "Flextronics International",
        "number_of_employees": 200000
    },
    {
        "name": "Safeway",
        "number_of_employees": 186000
    }
]


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [35]:
res = list(collection.find({'founded_month': { '$gte': 7 }} ).limit(1000))

print(len(res))
print(res[0])

1000
{'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'name': 'Wetpaint', 'permalink': 'abc2', 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint', 'homepage_url': 'http://wetpaint-inc.com', 'blog_url': 'http://digitalquarters.net/', 'blog_feed_url': 'http://digitalquarters.net/feed/', 'twitter_username': 'BachelrWetpaint', 'category_code': 'web', 'number_of_employees': 47, 'founded_year': 2005, 'founded_month': 10, 'founded_day': 17, 'deadpooled_year': 1, 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system', 'alias_list': '', 'email_address': 'info@wetpaint.com', 'phone_number': '206.859.6300', 'description': 'Technology Platform Company', 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27), 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013', 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for digital publishers

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [36]:
proj = {'_id': 0,'name': 1, 'founded_year': 1, 'acquisition': 1}
res = list(collection.find( {'acquisition': {'$exists': 'true' },
                             'acquisition.price_amount': {'$exists': 'true' , '$type': 'int', '$gte': 1000000},
                             'founded_year': { '$lt': 2000 } }, proj ))


print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))

196
{
    "acquisition": {
        "acquired_day": 9,
        "acquired_month": 7,
        "acquired_year": 2007,
        "acquiring_company": {
            "name": "Google",
            "permalink": "google"
        },
        "price_amount": 625000000,
        "price_currency_code": "USD",
        "source_description": "http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/",
        "source_url": "http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/",
        "term_code": "cash"
    },
    "founded_year": 1999,
    "name": "Postini"
}


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [37]:
proj = {'_id': 0,'name': 1, 'acquisition': 1}
res = list(collection.find( {'acquisition': {'$exists': 'true' },
                             'acquisition.acquired_year': {'$exists': 'true' , '$type': 'int', '$gte': 2010}},
                             proj ).sort('acquisition.price_amount', 1))


print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))

1100
{
    "acquisition": {
        "acquired_day": 28,
        "acquired_month": 11,
        "acquired_year": 2012,
        "acquiring_company": {
            "name": "MyHeritage",
            "permalink": "myheritage"
        },
        "price_amount": null,
        "price_currency_code": "USD",
        "source_description": "MyHeritage acquires Geni and $25M to build family tree of the whole world",
        "source_url": "http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/",
        "term_code": null
    },
    "name": "Geni"
}


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [48]:
proj = {'_id': 0,'name': 1, 'founded_year': 1}
res = list(collection.find({'founded_year' : { '$exists': 'true' }},proj).sort('founded_year', 1))

print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))

18801
{
    "founded_year": null,
    "name": "Flektor"
}


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [39]:
proj = {'_id': 0,'name': 1, 'founded_day': 1, 'acquisition': 1}
res = list(collection.find({'founded_day': { '$lte': 7 } },proj ).sort('acquisition.price_amount', -1).limit(10))


print(len(res))
print(json.dumps(res, indent=4, sort_keys=True))

10
[
    {
        "acquisition": {
            "acquired_day": 28,
            "acquired_month": 11,
            "acquired_year": 1998,
            "acquiring_company": {
                "name": "AOL",
                "permalink": "aol"
            },
            "price_amount": 4200000000,
            "price_currency_code": "USD",
            "source_description": "AOL buys Netscape for $4.2 billion",
            "source_url": "http://news.cnet.com/2100-1023-218360.html",
            "term_code": "stock"
        },
        "founded_day": 4,
        "name": "Netscape"
    },
    {
        "acquisition": {
            "acquired_day": 8,
            "acquired_month": 7,
            "acquired_year": 2002,
            "acquiring_company": {
                "name": "eBay",
                "permalink": "ebay"
            },
            "price_amount": 1500000000,
            "price_currency_code": "USD",
            "source_description": "eBay picks up PayPal for $1.5 billion",
            

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [40]:
proj = {'_id': 0,'name': 1, 'category_code': 1, 'number_of_employees': 1}
res = list(collection.find({'category_code': 'web', 
                            'number_of_employees': { '$gte': 4000 } },proj ).sort('number_of_employees',1))

print(len(res))
print(json.dumps(res, indent=4, sort_keys=True))

9
[
    {
        "category_code": "web",
        "name": "Expedia",
        "number_of_employees": 4400
    },
    {
        "category_code": "web",
        "name": "AOL",
        "number_of_employees": 8000
    },
    {
        "category_code": "web",
        "name": "Webkinz",
        "number_of_employees": 8657
    },
    {
        "category_code": "web",
        "name": "Rakuten",
        "number_of_employees": 10000
    },
    {
        "category_code": "web",
        "name": "Los Angeles Times Media Group",
        "number_of_employees": 10000
    },
    {
        "category_code": "web",
        "name": "Groupon",
        "number_of_employees": 10000
    },
    {
        "category_code": "web",
        "name": "Yahoo!",
        "number_of_employees": 13600
    },
    {
        "category_code": "web",
        "name": "eBay",
        "number_of_employees": 15000
    },
    {
        "category_code": "web",
        "name": "Experian",
        "number_of_employees": 15500
    }
]


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [41]:
proj = {'_id': 0,'name': 1, 'acquisition': 1}
res = list(collection.find({'acquisition.price_currency_code': 'EUR',
                            'acquisition': {'$exists': 'true' },
                            'acquisition.price_amount': {'$exists': 'true' , '$type': 'int', '$gte': 1000000}
                             }, proj ))


print(len(res))
print(json.dumps(res, indent=4, sort_keys=True))


11
[
    {
        "acquisition": {
            "acquired_day": 16,
            "acquired_month": 5,
            "acquired_year": 2008,
            "acquiring_company": {
                "name": "Vodafone",
                "permalink": "vodafone"
            },
            "price_amount": 31500000,
            "price_currency_code": "EUR",
            "source_description": "Danish Network ZYB Acquired By Vodafone For \u00e2\u201a\u00ac31.5 million",
            "source_url": "http://www.techcrunch.com/2008/05/16/danish-mobile-social-network-zyb-acquired-by-vodafone-for-e315-million/",
            "term_code": "cash"
        },
        "name": "ZYB"
    },
    {
        "acquisition": {
            "acquired_day": 3,
            "acquired_month": 1,
            "acquired_year": 2008,
            "acquiring_company": {
                "name": "Nokia Siemens Networks",
                "permalink": "nokia-siemens-networks"
            },
            "price_amount": 140000000,
            "

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [42]:
proj = {'_id': 0,'name': 1, 'acquisition': 1}
res = list(collection.find({'acquisition': {'$exists': 'true' },
                            'acquisition.acquired_month': {'$exists': 'true' , '$type': 'int', '$lte': 3}
                             }, proj ).limit(10))

print(len(res))
print(json.dumps(res, indent=4, sort_keys=True))

10
[
    {
        "acquisition": {
            "acquired_day": 31,
            "acquired_month": 1,
            "acquired_year": 2011,
            "acquiring_company": {
                "name": "KIT digital",
                "permalink": "kit-digital"
            },
            "price_amount": null,
            "price_currency_code": "USD",
            "source_description": "KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million",
            "source_url": "http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/",
            "term_code": null
        },
        "name": "Kyte"
    },
    {
        "acquisition": {
            "acquired_day": null,
            "acquired_month": 2,
            "acquired_year": 2007,
            "acquiring_company": {
                "name": "Nielsen",
                "permalink": "nielsen"
            },
            "price_amount": 327000000,
            "price_currency_code": "USD",
         

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [43]:
proj = {'_id': 0,'name': 1, 'founded_year': 1, 'acquisition': 1}
res = list(collection.find( { 'founded_year': { '$gte': 2000, '$lte': 2010 },
                            'acquisition': {'$exists': 'true' },
                            'acquisition.acquired_year': {'$exists': 'true' , '$type': 'int', '$gte': 2011}
                            },proj ))


print(len(res))
print(json.dumps(res[0], indent=4, sort_keys=True))

486
{
    "acquisition": {
        "acquired_day": 16,
        "acquired_month": 12,
        "acquired_year": 2013,
        "acquiring_company": {
            "name": "Viggle",
            "permalink": "viggle"
        },
        "price_amount": 30000000,
        "price_currency_code": "USD",
        "source_description": " Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint",
        "source_url": "http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet",
        "term_code": "cash_and_stock"
    },
    "founded_year": 2005,
    "name": "Wetpaint"
}


### 20. All the companies that have been 'deadpooled' after the third year.

In [96]:
proj = {'_id': 0,'name': 1, 'founded_year':1, 'deadpooled_year': 1}
res = list(collection.find({'founded_year': {'$exists': True  },'deadpooled_year': { '$exists': True } }, proj ))
res1 = list()

for element in res:
    
    if type(element['deadpooled_year']) == int and type(element['founded_year']) == int:
        subs = element['deadpooled_year'] - element['founded_year']
        
        if subs >= 3:
            res1.append(element)
            
print(len(res1))
print(res1[0])          


541
{'name': 'Omnidrive', 'founded_year': 2005, 'deadpooled_year': 2008}
